In [2]:
!pip install plyfile

In [6]:
import torch.nn as nn
import torchvision.models as models  # also needed for resnet18

class PoseRegressionNet(nn.Module):
    def __init__(self, pretrained=True):
        super(PoseRegressionNet, self).__init__()

        # 🔹 Load pretrained ResNet18 and remove its classifier
        resnet = models.resnet18(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])  # Remove final FC layer

        # 🔹 Add a new regression head: input 512-d → output 12-d
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 12)  # Output: x, y, z, r11..r33
        )

    def forward(self, x):
        features = self.backbone(x)     # Shape: [B, 512, 1, 1]
        output = self.regressor(features)  # Shape: [B, 12]
        return output

In [10]:
import os
import torch
import numpy as np
import pandas as pd
from plyfile import PlyData
from torchvision import transforms
from PIL import Image

# Load your model
model = PoseRegressionNet()
model.load_state_dict(torch.load('pose_regression_model.pth', map_location='cpu'))
model.eval()

# Load CSV
df = pd.read_csv('pose_labels.csv')

print(df[['class_id']].drop_duplicates().sort_values(by='class_id'))

# Same transforms used during training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Helpers
def load_ply_model(ply_path):
    ply = PlyData.read(ply_path)
    v = ply['vertex']
    return np.stack([v['x'], v['y'], v['z']], axis=-1).astype(np.float32)

def transform_points(pts, R, t):
    return pts @ R.T + t

def compute_add(gt_pts, pred_pts):
    return np.mean(np.linalg.norm(gt_pts - pred_pts, axis=1))

# Run ADD evaluation
add_scores = []
PLY_FOLDER = '/home/erythm/6d/dataset/Linemod_preprocessed/models/'  # ✅ YOUR path

for idx, row in df.iterrows():
    img_path = row['crop_path']
    class_id = int(row['class_id'])

    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)

    with torch.no_grad():
        pred_pose = model(img_tensor).squeeze().numpy()

    pred_t = pred_pose[:3]
    pred_R = np.array(pred_pose[3:]).reshape(3, 3)

    gt_t = np.array([row['x'], row['y'], row['z']])
    gt_R = np.array([row[f'R{i}'] for i in range(1, 10)]).reshape(3, 3)

    ply_path = os.path.join(PLY_FOLDER, f"obj_{class_id:02d}.ply")
    if not os.path.exists(ply_path):
        print(f"[SKIP] No .ply for class {class_id}")
        continue

    pts = load_ply_model(ply_path)
    gt_pts = transform_points(pts, gt_R, gt_t)
    pred_pts = transform_points(pts, pred_R, pred_t)

    add = compute_add(gt_pts, pred_pts)
    add_scores.append(add)

print("\n✅ ADD Evaluation Complete — Average ADD: {:.6f} meters".format(np.mean(add_scores)))

    class_id
17         1
3          2
13         4
5          5
8          6
2          8
6          9
0         10
1         11
12        12
11        13


/home/erythm/anaconda3/envs/6d/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/erythm/anaconda3/envs/6d/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



✅ ADD Evaluation Complete — Average ADD: 4.593893 meters
